Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in 1_notmnist.ipynb.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [19]:
batch_size = 128
l2_loss_layer1 = 0.05
# l2_loss_layer2 = 1.

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size*image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + \
    l2_loss_layer1 * tf.nn.l2_loss(weights) + l2_loss_layer1 * tf.nn.l2_loss(biases)
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [20]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]   
        
        feed_dict = {tf_train_dataset: batch_data,
                     tf_train_labels: batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy training accuracy: %.1f%%' % accuracy(
                    predictions, batch_labels))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 172.144516
Minibatch accuracy training accuracy: 9.4%
Validation accuracy: 9.8%
Minibatch loss at step 500: 1.150087
Minibatch accuracy training accuracy: 70.3%
Validation accuracy: 75.4%
Minibatch loss at step 1000: 1.155479
Minibatch accuracy training accuracy: 75.0%
Validation accuracy: 80.2%
Minibatch loss at step 1500: 0.994447
Minibatch accuracy training accuracy: 77.3%
Validation accuracy: 79.1%
Minibatch loss at step 2000: 1.020738
Minibatch accuracy training accuracy: 76.6%
Validation accuracy: 76.6%
Minibatch loss at step 2500: 1.170055
Minibatch accuracy training accuracy: 71.1%
Validation accuracy: 80.4%
Minibatch loss at step 3000: 0.905873
Minibatch accuracy training accuracy: 83.6%
Validation accuracy: 77.3%
Test accuracy: 83.0%


In [27]:
batch_size = 128
hidden_layer_size = 1024
l2_loss_layer1 = 0.001
l2_loss_layer2 = 0.005

graph = tf.Graph()
with graph.as_default():
    
    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, 
                                     shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    biases = tf.Variable(tf.zeros([hidden_layer_size]))
    
    weights2 = tf.Variable(
        tf.truncated_normal([hidden_layer_size, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits_pt1 = tf.matmul(tf_train_dataset, weights) + biases
    relu_out = tf.nn.relu(logits_pt1)
    logits = tf.matmul(relu_out, weights2) + biases2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + \
        l2_loss_layer1 * tf.nn.l2_loss(weights) + l2_loss_layer1 * tf.nn.l2_loss(biases) + \
        l2_loss_layer2 * tf.nn.l2_loss(weights2) + l2_loss_layer2 * tf.nn.l2_loss(biases2)
    
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_intermed = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)    
    valid_prediction = tf.nn.softmax(
        tf.matmul(valid_intermed, weights2) + biases2)
    test_intermed = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(test_intermed, weights2) + biases2)    

In [28]:
num_steps = 6001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]   
        
        feed_dict = {tf_train_dataset: batch_data,
                     tf_train_labels: batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy training accuracy: %.1f%%' % accuracy(
                    predictions, batch_labels))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 688.449707
Minibatch accuracy training accuracy: 6.2%
Validation accuracy: 33.0%
Minibatch loss at step 500: 271.663116
Minibatch accuracy training accuracy: 73.4%
Validation accuracy: 78.4%
Minibatch loss at step 1000: 213.305588
Minibatch accuracy training accuracy: 76.6%
Validation accuracy: 82.2%
Minibatch loss at step 1500: 172.228302
Minibatch accuracy training accuracy: 82.0%
Validation accuracy: 82.6%
Minibatch loss at step 2000: 143.935577
Minibatch accuracy training accuracy: 75.0%
Validation accuracy: 78.5%
Minibatch loss at step 2500: 115.391907
Minibatch accuracy training accuracy: 77.3%
Validation accuracy: 82.2%
Minibatch loss at step 3000: 93.027939
Minibatch accuracy training accuracy: 85.9%
Validation accuracy: 83.9%
Minibatch loss at step 3500: 76.213005
Minibatch accuracy training accuracy: 87.5%
Validation accuracy: 79.8%
Minibatch loss at step 4000: 62.635403
Minibatch accuracy training accuracy: 80.5%
Validation accuracy: 80.

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [39]:
batch_size = 128
hidden_layer_size = 1024
l2_loss_layer1 = 0.
l2_loss_layer2 = 0.

graph = tf.Graph()
with graph.as_default():
    
    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, 
                                     shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    biases = tf.Variable(tf.zeros([hidden_layer_size]))
    
    weights2 = tf.Variable(
        tf.truncated_normal([hidden_layer_size, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits_pt1 = tf.matmul(tf_train_dataset, weights) + biases
    relu_out = tf.nn.relu(logits_pt1)
    logits = tf.matmul(relu_out, weights2) + biases2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + \
        l2_loss_layer1 * tf.nn.l2_loss(weights) + l2_loss_layer1 * tf.nn.l2_loss(biases) + \
        l2_loss_layer2 * tf.nn.l2_loss(weights2) + l2_loss_layer2 * tf.nn.l2_loss(biases2)
    
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_intermed = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)    
    valid_prediction = tf.nn.softmax(
        tf.matmul(valid_intermed, weights2) + biases2)
    test_intermed = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(test_intermed, weights2) + biases2)    

In [40]:
num_steps = 6001
train_subset = train_dataset[:128*3, :]
train_subset_labels = train_labels[:128*3, :]
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_subset_labels.shape[0] - batch_size)
        batch_data = train_subset[offset:(offset + batch_size), :]
        batch_labels = train_subset_labels[offset:(offset + batch_size), :]   
        
        feed_dict = {tf_train_dataset: batch_data,
                     tf_train_labels: batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy training accuracy: %.1f%%' % accuracy(
                    predictions, batch_labels))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 380.817444
Minibatch accuracy training accuracy: 10.9%
Validation accuracy: 27.7%
Minibatch loss at step 500: 0.000000
Minibatch accuracy training accuracy: 100.0%
Validation accuracy: 71.2%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy training accuracy: 100.0%
Validation accuracy: 71.2%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy training accuracy: 100.0%
Validation accuracy: 71.2%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy training accuracy: 100.0%
Validation accuracy: 71.2%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy training accuracy: 100.0%
Validation accuracy: 71.2%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy training accuracy: 100.0%
Validation accuracy: 71.2%
Minibatch loss at step 3500: 0.000000
Minibatch accuracy training accuracy: 100.0%
Validation accuracy: 71.2%
Minibatch loss at step 4000: 0.000000
Minibatch accuracy training accuracy: 100.0%
Validation accuracy: 71.2%
M

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [36]:
batch_size = 128
hidden_layer_size = 1024
l2_loss_layer1 = 0.
l2_loss_layer2 = 0.
dropout_keep_prob = 0.5

graph = tf.Graph()
with graph.as_default():
    
    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, 
                                     shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    biases = tf.Variable(tf.zeros([hidden_layer_size]))
    
    weights2 = tf.Variable(
        tf.truncated_normal([hidden_layer_size, num_labels]))
    biases2 = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits_pt1 = tf.matmul(tf_train_dataset, weights) + biases
    relu_out = tf.nn.relu(logits_pt1)
    dropped_out_relu_out = tf.nn.dropout(relu_out, keep_prob=dropout_keep_prob)
    logits = tf.matmul(dropped_out_relu_out, weights2) + biases2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + \
        l2_loss_layer1 * tf.nn.l2_loss(weights) + l2_loss_layer1 * tf.nn.l2_loss(biases) + \
        l2_loss_layer2 * tf.nn.l2_loss(weights2) + l2_loss_layer2 * tf.nn.l2_loss(biases2)
    
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.2).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    
    # accuracy figures as we train.
    train_intermed = tf.nn.relu(tf.matmul(tf_train_dataset, weights) + biases)    
    train_prediction = tf.nn.softmax(
        tf.matmul(train_intermed, weights2) + biases2)
    valid_intermed = tf.nn.relu(tf.matmul(tf_valid_dataset, weights) + biases)    
    valid_prediction = tf.nn.softmax(
        tf.matmul(valid_intermed, weights2) + biases2)
    test_intermed = tf.nn.relu(tf.matmul(tf_test_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(test_intermed, weights2) + biases2)    

In [38]:
num_steps = 6001
train_subset = train_dataset[:128*3, :]
train_subset_labels = train_labels[:128*3, :]
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_subset_labels.shape[0] - batch_size)
        batch_data = train_subset[offset:(offset + batch_size), :]
        batch_labels = train_subset_labels[offset:(offset + batch_size), :]   
        
        feed_dict = {tf_train_dataset: batch_data,
                     tf_train_labels: batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy training accuracy: %.1f%%' % accuracy(
                    predictions, batch_labels))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 445.942871
Minibatch accuracy training accuracy: 14.8%
Validation accuracy: 28.0%
Minibatch loss at step 500: 1.908525
Minibatch accuracy training accuracy: 100.0%
Validation accuracy: 75.2%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy training accuracy: 100.0%
Validation accuracy: 75.4%
Minibatch loss at step 1500: 0.592286
Minibatch accuracy training accuracy: 100.0%
Validation accuracy: 75.4%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy training accuracy: 100.0%
Validation accuracy: 75.7%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy training accuracy: 100.0%
Validation accuracy: 75.6%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy training accuracy: 100.0%
Validation accuracy: 75.9%
Minibatch loss at step 3500: 0.000000
Minibatch accuracy training accuracy: 100.0%
Validation accuracy: 75.7%
Minibatch loss at step 4000: 0.000000
Minibatch accuracy training accuracy: 100.0%
Validation accuracy: 75.7%
M

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [48]:
28*28

784

In [54]:
train_dataset.max()

0.5

In [55]:
train_dataset.min()

-0.5

In [222]:
# batch_size = 128
# hidden_layers = [1024, 1024, 512, 512]
# l2_losses = [0.01, 0.01, 0.01, 0.01, 0.01]  # one more than hidden layers
# dropout_keep_probs = [1., 0.5, 0.5, 0.5, 0.5]  # one more than hidden layers
# gradient_descent_rate = 0.01
batch_size = 128
hidden_layers = [2086]
# l2_losses = [0.00001, 0.00001]  # one more than hidden layers
l2_losses = [0., 0.]
# dropout_keep_probs = [0.8, 0.5, 0.5, 0.5]  # one more than hidden layers
dropout_keep_probs = [1.0, 0.6]
gradient_descent_rate = 0.03
decay_rate = 0.98

graph = tf.Graph()
with graph.as_default():
    
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(gradient_descent_rate, global_step, decay_rate=decay_rate, decay_steps=300000)
    
    weights = []
    biases = []
    last_layer_size = image_size * image_size
    for i in range(len(hidden_layers)):
        weights.append(tf.Variable(tf.truncated_normal([last_layer_size, hidden_layers[i]]) / (2)))
        biases.append(tf.Variable(tf.zeros([hidden_layers[i]])))
        last_layer_size = hidden_layers[i]
        
    weights.append(tf.Variable(tf.truncated_normal([last_layer_size, num_labels])))
    biases.append(tf.Variable(tf.zeros([num_labels])))
    
    last_layer_act = tf.nn.dropout(tf_train_dataset, dropout_keep_probs[0])
#     last_layer_logits = 
    for i in range(len(hidden_layers)):
        layer_logits = tf.matmul(last_layer_act, weights[i]) + biases[i]
        layer_act = tf.nn.relu(layer_logits)
        layer_drop = tf.nn.dropout(layer_act, keep_prob=dropout_keep_probs[i+1])
        last_layer_act = layer_drop
        
    logits = tf.matmul(last_layer_act, weights[-1]) + biases[-1]
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    for i in range(len(hidden_layers)):
        loss += l2_losses[i] * tf.nn.l2_loss(weights[i]) + l2_losses[i] * tf.nn.l2_loss(biases[i])
    loss += l2_losses[-1] * tf.nn.l2_loss(weights[-1]) + l2_losses[-1] * tf.nn.l2_loss(biases[-1])
        
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    last_layer_act = tf_train_dataset
    for i in range(len(hidden_layers)):
        layer_logits = tf.matmul(last_layer_act, weights[i]) + biases[i]
        layer_act = tf.nn.relu(layer_logits)
        last_layer_act = layer_act
    logits = tf.matmul(last_layer_act, weights[-1]) + biases[-1]
    train_prediction = tf.nn.softmax(logits)
    
    last_layer_act = valid_dataset
    for i in range(len(hidden_layers)):
        layer_logits = tf.matmul(last_layer_act, weights[i]) + biases[i]
        layer_act = tf.nn.relu(layer_logits)
        last_layer_act = layer_act
    logits = tf.matmul(last_layer_act, weights[-1]) + biases[-1]
    valid_prediction = tf.nn.softmax(logits)
    
    last_layer_act = test_dataset
    for i in range(len(hidden_layers)):
        layer_logits = tf.matmul(last_layer_act, weights[i]) + biases[i]
        layer_act = tf.nn.relu(layer_logits)
        last_layer_act = layer_act
    logits = tf.matmul(last_layer_act, weights[-1]) + biases[-1]
    test_prediction = tf.nn.softmax(logits)

In [223]:
num_steps = 6001
# train_subset = train_dataset[:128*3, :]
# train_subset_labels = train_labels[:128*3, :]
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_dataset.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]   
        
        feed_dict = {tf_train_dataset: batch_data,
                     tf_train_labels: batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 300 == 0):
#             all_of_weights = np.concatenate([_w.eval().flatten() for _w in weights])
#             eval_weights = [_w.eval() for _w in weights]
#             print('Weights median: {}'.format([np.median(_w) for _w in eval_weights]))
#             print('Weights max: {}'.format([np.max(_w) for _w in eval_weights]))
#             print('Learn rate: {}'.format(learning_rate.eval()))
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy training accuracy: %.1f%%' % accuracy(
                    predictions, batch_labels))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 349.051239
Minibatch accuracy training accuracy: 6.2%
Validation accuracy: 22.2%
Minibatch loss at step 300: 48.201424
Minibatch accuracy training accuracy: 78.9%
Validation accuracy: 82.3%
Minibatch loss at step 600: 29.676746
Minibatch accuracy training accuracy: 82.8%
Validation accuracy: 84.2%
Minibatch loss at step 900: 44.318794
Minibatch accuracy training accuracy: 82.0%
Validation accuracy: 85.1%
Minibatch loss at step 1200: 26.554981
Minibatch accuracy training accuracy: 82.0%
Validation accuracy: 85.3%
Minibatch loss at step 1500: 30.843069
Minibatch accuracy training accuracy: 85.2%
Validation accuracy: 85.8%
Minibatch loss at step 1800: 14.047193
Minibatch accuracy training accuracy: 84.4%
Validation accuracy: 86.3%
Minibatch loss at step 2100: 21.731916
Minibatch accuracy training accuracy: 86.7%
Validation accuracy: 86.1%
Minibatch loss at step 2400: 20.387182
Minibatch accuracy training accuracy: 85.9%
Validation accuracy: 86.3%
Mini

In [262]:
# batch_size = 128
# hidden_layers = [1024, 1024, 512, 512]
# l2_losses = [0.01, 0.01, 0.01, 0.01, 0.01]  # one more than hidden layers
# dropout_keep_probs = [1., 0.5, 0.5, 0.5, 0.5]  # one more than hidden layers
# gradient_descent_rate = 0.01
batch_size = 50
hidden_layers = [1800, 1028]
# l2_losses = [0.00001, 0.00001]  # one more than hidden layers
l2_losses = [0., 0., 0.00001]
# dropout_keep_probs = [0.8, 0.5, 0.5, 0.5]  # one more than hidden layers
dropout_keep_probs = [1.0, 0.6, 1.0]
gradient_descent_rate = 0.00008
decay_rate = 1.0
momentum = 0.7

graph = tf.Graph()
with graph.as_default():
    
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(gradient_descent_rate, global_step, decay_rate=decay_rate, decay_steps=300000)
    
    weights = []
    biases = []
    last_layer_size = image_size * image_size
    for i in range(len(hidden_layers)):
        weights.append(tf.Variable(tf.truncated_normal([last_layer_size, hidden_layers[i]]) / (2)))
        biases.append(tf.Variable(tf.zeros([hidden_layers[i]])))
        last_layer_size = hidden_layers[i]
        
    weights.append(tf.Variable(tf.truncated_normal([last_layer_size, num_labels])))
    biases.append(tf.Variable(tf.zeros([num_labels])))
    
    last_layer_act = tf.nn.dropout(tf_train_dataset, dropout_keep_probs[0])
#     last_layer_logits = 
    for i in range(len(hidden_layers)):
        layer_logits = tf.matmul(last_layer_act, weights[i]) + biases[i]
        layer_act = tf.nn.relu(layer_logits)
        layer_drop = tf.nn.dropout(layer_act, keep_prob=dropout_keep_probs[i+1])
        last_layer_act = layer_drop
        
    logits = tf.matmul(last_layer_act, weights[-1]) + biases[-1]
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    for i in range(len(hidden_layers)):
        loss += l2_losses[i] * tf.nn.l2_loss(weights[i]) + l2_losses[i] * tf.nn.l2_loss(biases[i])
    loss += l2_losses[-1] * tf.nn.l2_loss(weights[-1]) + l2_losses[-1] * tf.nn.l2_loss(biases[-1])
    
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum).minimize(loss, global_step=global_step)
    
    last_layer_act = tf_train_dataset
    for i in range(len(hidden_layers)):
        layer_logits = tf.matmul(last_layer_act, weights[i]) + biases[i]
        layer_act = tf.nn.relu(layer_logits)
        last_layer_act = layer_act
    logits = tf.matmul(last_layer_act, weights[-1]) + biases[-1]
    train_prediction = tf.nn.softmax(logits)
    
    last_layer_act = valid_dataset
    for i in range(len(hidden_layers)):
        layer_logits = tf.matmul(last_layer_act, weights[i]) + biases[i]
        layer_act = tf.nn.relu(layer_logits)
        last_layer_act = layer_act
    logits = tf.matmul(last_layer_act, weights[-1]) + biases[-1]
    valid_prediction = tf.nn.softmax(logits)
    
    last_layer_act = test_dataset
    for i in range(len(hidden_layers)):
        layer_logits = tf.matmul(last_layer_act, weights[i]) + biases[i]
        layer_act = tf.nn.relu(layer_logits)
        last_layer_act = layer_act
    logits = tf.matmul(last_layer_act, weights[-1]) + biases[-1]
    test_prediction = tf.nn.softmax(logits)

In [264]:
num_steps = 60001
# train_subset = train_dataset[:128*3, :]
# train_subset_labels = train_labels[:128*3, :]
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_dataset.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]   
        
        feed_dict = {tf_train_dataset: batch_data,
                     tf_train_labels: batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
#             all_of_weights = np.concatenate([_w.eval().flatten() for _w in weights])
#             eval_weights = [_w.eval() for _w in weights]
#             print('Weights median: {}'.format([np.median(_w) for _w in eval_weights]))
#             print('Weights max: {}'.format([np.max(_w) for _w in eval_weights]))
#             print('Learn rate: {}'.format(learning_rate.eval()))
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy training accuracy: %.1f%%' % accuracy(
                    predictions, batch_labels))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2088.052734
Minibatch accuracy training accuracy: 10.0%
Validation accuracy: 12.9%
Minibatch loss at step 500: 504.614471
Minibatch accuracy training accuracy: 66.0%
Validation accuracy: 77.9%
Minibatch loss at step 1000: 258.936188
Minibatch accuracy training accuracy: 80.0%
Validation accuracy: 80.3%
Minibatch loss at step 1500: 176.297821
Minibatch accuracy training accuracy: 90.0%
Validation accuracy: 81.4%
Minibatch loss at step 2000: 193.179550
Minibatch accuracy training accuracy: 88.0%
Validation accuracy: 82.1%
Minibatch loss at step 2500: 109.900932
Minibatch accuracy training accuracy: 92.0%
Validation accuracy: 82.5%
Minibatch loss at step 3000: 211.127960
Minibatch accuracy training accuracy: 86.0%
Validation accuracy: 82.6%
Minibatch loss at step 3500: 237.889877
Minibatch accuracy training accuracy: 76.0%
Validation accuracy: 82.8%
Minibatch loss at step 4000: 213.846939
Minibatch accuracy training accuracy: 80.0%
Validation accuracy

In [279]:
# batch_size = 128
# hidden_layers = [1024, 1024, 512, 512]
# l2_losses = [0.01, 0.01, 0.01, 0.01, 0.01]  # one more than hidden layers
# dropout_keep_probs = [1., 0.5, 0.5, 0.5, 0.5]  # one more than hidden layers
# gradient_descent_rate = 0.01
batch_size = 20
hidden_layers = [800, 800]
# l2_losses = [0.00001, 0.00001]  # one more than hidden layers
l2_losses = [0., 0., 0.00001]
# dropout_keep_probs = [0.8, 0.5, 0.5, 0.5]  # one more than hidden layers
dropout_keep_probs = [1.0, 0.6, 1.0]
gradient_descent_rate = 0.00008
decay_rate = 1.0
momentum = 0.0

graph = tf.Graph()
with graph.as_default():
    
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(gradient_descent_rate, global_step, decay_rate=decay_rate, decay_steps=300000)
    
    weights = []
    biases = []
    last_layer_size = image_size * image_size
    for i in range(len(hidden_layers)):
        weights.append(tf.Variable(tf.truncated_normal([last_layer_size, hidden_layers[i]], stddev=0.1)))
        biases.append(tf.Variable(tf.zeros([hidden_layers[i]])))
        last_layer_size = hidden_layers[i]
        
    weights.append(tf.Variable(tf.truncated_normal([last_layer_size, num_labels])))
    biases.append(tf.Variable(tf.zeros([num_labels])))
    
    last_layer_act = tf.nn.dropout(tf_train_dataset, dropout_keep_probs[0])
#     last_layer_logits = 
    for i in range(len(hidden_layers)):
        layer_logits = tf.matmul(last_layer_act, weights[i]) + biases[i]
        layer_act = tf.nn.relu(layer_logits)
        layer_drop = tf.nn.dropout(layer_act, keep_prob=dropout_keep_probs[i+1])
        last_layer_act = layer_drop
        
    logits = tf.matmul(last_layer_act, weights[-1]) + biases[-1]
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    for i in range(len(hidden_layers)):
        loss += l2_losses[i] * tf.nn.l2_loss(weights[i]) + l2_losses[i] * tf.nn.l2_loss(biases[i])
    loss += l2_losses[-1] * tf.nn.l2_loss(weights[-1]) + l2_losses[-1] * tf.nn.l2_loss(biases[-1])
    
    optimizer = tf.train.MomentumOptimizer(learning_rate, momentum).minimize(loss, global_step=global_step)
    
    last_layer_act = tf_train_dataset
    for i in range(len(hidden_layers)):
        layer_logits = tf.matmul(last_layer_act, weights[i]) + biases[i]
        layer_act = tf.nn.relu(layer_logits)
        last_layer_act = layer_act
    logits = tf.matmul(last_layer_act, weights[-1]) + biases[-1]
    train_prediction = tf.nn.softmax(logits)
    
    last_layer_act = valid_dataset
    for i in range(len(hidden_layers)):
        layer_logits = tf.matmul(last_layer_act, weights[i]) + biases[i]
        layer_act = tf.nn.relu(layer_logits)
        last_layer_act = layer_act
    logits = tf.matmul(last_layer_act, weights[-1]) + biases[-1]
    valid_prediction = tf.nn.softmax(logits)
    
    last_layer_act = test_dataset
    for i in range(len(hidden_layers)):
        layer_logits = tf.matmul(last_layer_act, weights[i]) + biases[i]
        layer_act = tf.nn.relu(layer_logits)
        last_layer_act = layer_act
    logits = tf.matmul(last_layer_act, weights[-1]) + biases[-1]
    test_prediction = tf.nn.softmax(logits)

In [280]:
num_steps = 12001
# train_subset = train_dataset[:128*3, :]
# train_subset_labels = train_labels[:128*3, :]
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_dataset.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]   
        
        feed_dict = {tf_train_dataset: batch_data,
                     tf_train_labels: batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
#             all_of_weights = np.concatenate([_w.eval().flatten() for _w in weights])
#             eval_weights = [_w.eval() for _w in weights]
#             print('Weights median: {}'.format([np.median(_w) for _w in eval_weights]))
#             print('Weights max: {}'.format([np.max(_w) for _w in eval_weights]))
#             print('Learn rate: {}'.format(learning_rate.eval()))
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy training accuracy: %.1f%%' % accuracy(
                    predictions, batch_labels))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(
                    valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 70.160248
Minibatch accuracy training accuracy: 20.0%
Validation accuracy: 13.0%
Minibatch loss at step 500: 25.553854
Minibatch accuracy training accuracy: 70.0%
Validation accuracy: 65.4%
Minibatch loss at step 1000: 11.464728
Minibatch accuracy training accuracy: 60.0%
Validation accuracy: 72.8%
Minibatch loss at step 1500: 10.671533
Minibatch accuracy training accuracy: 85.0%
Validation accuracy: 75.2%
Minibatch loss at step 2000: 18.406536
Minibatch accuracy training accuracy: 65.0%
Validation accuracy: 76.6%
Minibatch loss at step 2500: 13.140364
Minibatch accuracy training accuracy: 80.0%
Validation accuracy: 77.5%
Minibatch loss at step 3000: 18.952641
Minibatch accuracy training accuracy: 75.0%
Validation accuracy: 78.3%
Minibatch loss at step 3500: 18.790234
Minibatch accuracy training accuracy: 65.0%
Validation accuracy: 79.0%
Minibatch loss at step 4000: 5.510175
Minibatch accuracy training accuracy: 90.0%
Validation accuracy: 79.0%
Min